### Merge and combine yearly pitching and batting data

In [112]:
import pandas as pd
import numpy as np

df_list = []
for year in range(2011, 2026):

    df = pd.read_csv(f'stan_batting/stan_batting_{year}.csv', skiprows=1, encoding='utf-8')
    df['Year'] = year
    df_list.append(df)

merged_batting_df = pd.concat(df_list, ignore_index=True)
merged_batting_df.to_csv('stan_batting/merged_stan_batting.csv', index=False)

df_list = []
for year in range(2011, 2026):

    df = pd.read_csv(f'stan_pitching/stan_pitching_{year}.csv', skiprows=1, encoding='utf-8')
    df_list.append(df)

merged_pitching_df = pd.concat(df_list, ignore_index=True)
merged_pitching_df.to_csv('stan_pitching/merged_stan_pitching.csv', index=False)

# Concatenate two dataframes with different columns, filling missing columns with NaN
concatenated_df = pd.concat([merged_batting_df, merged_pitching_df], ignore_index=True, sort=False)
concatenated_df.rename(columns={'Player': 'Name'}, inplace=True) # rename Player to Name for consistency
concatenated_df.dropna(subset=['Team'], inplace=True) # drop rows where team is NaN
concatenated_df.to_csv('merged_batting_pitching.csv', index=False)
concatenated_df

,Rk,Name,Age,Team,Lg,WAR,G,PA,AB,R,...,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/BB
0,1.0,Jacoby Ellsbury*,27.0,BOS,AL,8.3,158.0,732.0,660.0,119.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Dustin Pedroia,27.0,BOS,AL,8.0,159.0,731.0,635.0,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Ian Kinsler,29.0,TEX,AL,7.0,155.0,723.0,620.0,121.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,Michael Bourn*,28.0,2TM,NL,3.0,158.0,722.0,656.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,Michael Bourn*,28.0,HOU,NL,2.2,105.0,473.0,429.0,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32990,856.0,Austin Slater,32.0,NYY,AL,0.0,1.0,NaN,NaN,0.0,...,0.0,3.0,NaN,3.15,1.5,13.5,0.0,0.0,0.0,NaN
32991,857.0,Luis Torrens,29.0,NYM,NL,-0.2,2.0,NaN,NaN,4.0,...,0.0,8.0,12.0,46.65,9.0,67.5,27.0,13.5,0.0,0.0
32992,858.0,Carlos Durán,23.0,ATH,AL,-0.2,1.0,NaN,NaN,3.0,...,0.0,5.0,11.0,30.15,12.0,27.0,0.0,81.0,0.0,0.0
32993,859.0,Nicky Lopez,30.0,LAA,AL,0.0,1.0,NaN,NaN,0.0,...,0.0,2.0,NaN,12.15,3.0,0.0,0.0,27.0,0.0,0.0


### Build id to merge batting/pitching with salary

In [113]:
import unicodedata

salary_df = pd.read_csv('mlb_salary_data.csv', encoding='latin1')
player_data = pd.read_csv('merged_batting_pitching.csv', low_memory=False)
def normalize_name(name):
    # Normalize to NFKD and encode to ASCII, ignoring errors (removes accents)
    name =  unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8')
    name = name.lower().replace('.', '').replace("'", '').strip()
    return name

salary_df['Name'] = salary_df['Name'].apply(normalize_name)
player_data['Name'] = player_data['Name'].apply(normalize_name)


# create alteration function to get 1st letter of first name and first 3 letters of last name
def convert_name(row):
    parts = row['Name'].split(' ')
    max_idx = (min(2, len(parts[0])))  # Ensure at most 2 characters are taken
    part1 = parts[0][:max_idx]  # First letter of the first name
    part2 = ''
    last_name_len = '0'
    for part in parts[1:]:
        if (len(part)>=3) or (part == parts[-1]): # check that part is either final part or at least 3 characters long
            max_idx = min(4, len(part))  # Ensure at most 4 characters are taken
            part2 = part[:max_idx]  # First three letters of the last name
            last_name_len = str(len(part)) # Length of the last name part
            continue # only use the first valid last name part
    team = row['Team']  
    return part1 + part2 + last_name_len + team.lower()

salary_df['id'] = salary_df.apply(convert_name, axis=1)
player_data['id'] = player_data.apply(convert_name, axis=1)
# player_data

### Test ID

In [123]:
from collections import Counter

year = 2024 # limit to single year for testing
print('unique ids in salary: {}'.format(salary_df[salary_df['Year'] == year]['id'].nunique()))
print('unique ids in player_data: {}'.format(player_data[player_data['Year'] == year]['id'].nunique()))

# get df of ids that have multiple instances
def check_doubled_ids(df):
    dfs = []
    for year in range(2011, 2026):
        counter = Counter(df[df['Year'] == year]['id'])
        doubled_ids = [item for item, count in counter.items() if count > 1]
        dfs.append(df[df['id'].isin(doubled_ids) & (df['Year'] == year)])
    df = pd.concat(dfs)
    df.sort_values(by=['id', 'Salary'], ascending=[True, False], inplace=True)
    return df

# if there are two players with the same id keep the one with the higher salary
def keep_higher_salary(df):
    # count the ids that appear more than once in salary_df for the given year
    temp_df = check_doubled_ids(df.copy())
    drop_idxs = []
    for id in temp_df['id'].unique():
        names = temp_df[temp_df.id == id]
        drop_idxs.append(names.index[1:]) # keep the first instance, drop the rest

    drop_idxs_flat = [idx for sublist in drop_idxs for idx in sublist]
    df = df.drop(drop_idxs_flat)
    df = df.reset_index(drop=True)
    return df

keep_higher_salary(salary_df)

unique ids in salary: 1070
unique ids in player_data: 895


,Year,Team,Name,Salary,id
0,2011,ARI,kelly johnson,"$5,850,000",kejohn7ari
1,2011,ARI,joe saunders,"$5,500,000",josaun8ari
2,2011,ARI,chris young,"$5,200,000",chyoun5ari
3,2011,ARI,stephen drew,"$4,650,000",stdrew4ari
4,2011,ARI,justin upton,"$4,458,333",juupto5ari
...,...,...,...,...,...
13944,2024,COL,german marquez,"$10,000,000",gemarq7col
13945,2024,COL,daniel bard,"$9,500,000",dabard4col
13946,2024,COL,dakota hudson,"$1,500,000",dahuds6col
13947,2024,COL,tyler kinley,"$1,300,000",tykinl6col
